# List of Exposures  at USDF in OGA


- work with Weakly_2024_17
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/13
- last update : 2024/05/02



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2024_17/Spectractor/lib/python', '/home/d/dagoret/repos/repos_w_2024_17/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/ctrl_mpexec/g9f21ca10cf+81e60bdb09/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/obs_lsst/g5c5439fafc+e5b148903d/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/pipe_tasks/gc07e1c2157+b2dbe6b631/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/shapelet/ge79ae78c31+d6b5cd48b5/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/meas_modelfit/g591dd9f2cf+b2918d57ae/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/scarlet_lite/gee10cc3b42+a6fc98d2e7/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/proxmin/g33b4157f25/lib/python', '/opt/lsst/software/stack/stack/minicon

In [3]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_17/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_17/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_17/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2024_16
   g4213664e8e+c2911b1728 	current w_2024_16 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
#import lsst.afw.display as afwDisplay
#afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

ModuleNotFoundError: No module named 'lsst.utils'

In [ ]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
collection='LATISS/raw/all'

## List of Exposures

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [ ]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    try:
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd, info_timespan_begin_mjd  ]
 
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

In [ ]:
info

In [ ]:
df_exposure

In [ ]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [ ]:
df_exposure

## Selection of science

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']

In [ ]:
df_science.sort_values(by="id",ascending=True,inplace=True)

In [ ]:
df_science.head(60)

In [ ]:
df_science.tail(60)

In [ ]:
df_science.day_obs.unique()

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [ ]:
assert False

In [ ]:

# Januaryr 2024
#date_sel = 20240129
#date_sel = 20240130
#date_sel = 20240131
#date_sel = 20230928
date_sel = 202405

## Query Dimension records on Exposures

In [ ]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [ ]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [ ]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [ ]:
len(df_science_selected)

# List of filters

In [ ]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

In [ ]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

## Selection of filter

- But now we have a loop on filters

In [ ]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

In [ ]:
N = len(all_def_science_selected_final)

In [ ]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

In [ ]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [ ]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [ ]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [ ]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [ ]:
df_forsave = pd.concat(all_def_science_selected_final)

In [ ]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [ ]:
filename_out_csv

In [ ]:
df_forsave.to_csv(filename_out_csv) 

In [ ]:
#! pip install openpyxl

In [ ]:
df_forsave.to_excel(filename_out_excel) 